In [ ]:
#!/usr/bin/env python3
"""
ORCID 女性姓名变化检测程序
"""

import os
import csv
import logging
import multiprocessing as mp
from multiprocessing import Pool, Manager, Lock
from concurrent.futures import ProcessPoolExecutor, as_completed
import time
from pathlib import Path
from typing import Dict, List, Optional, Tuple
import signal
import sys
from functools import partial

from bs4 import BeautifulSoup
import gender_guesser.detector as gender

# 全局配置
CONFIG = {
    'base_path': '/hy-tmp/ORCID_2024_10_summaries',
    'output_file': '/hy-tmp/female_name_changes.csv',
    'log_file': '/hy-tmp/orcid_processing.log',
    'max_workers': 20,  # 保留2个核心给系统
    'chunk_size': 1000,  # 每个进程处理的文件数量
    'batch_size': 100,  # 批量写入CSV的记录数
}

class ORCIDProcessor:
    def __init__(self, base_path: str, output_file: str, max_workers: int = 38):
        self.base_path = Path(base_path)
        self.output_file = output_file
        self.max_workers = max_workers
        self.gender_detector = gender.Detector()
        self.setup_logging()
        
        # 统计信息
        self.stats = {
            'total_files': 0,
            'processed_files': 0,
            'female_records': 0,
            'name_changes': 0,
            'errors': 0
        }

    def setup_logging(self):
        """配置日志系统"""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(CONFIG['log_file']),
                logging.StreamHandler(sys.stdout)
            ]
        )
        self.logger = logging.getLogger(__name__)

    def extract_name_info(self, soup: BeautifulSoup) -> Optional[Dict]:
        """从BeautifulSoup对象中提取姓名信息和性别"""
        try:
            person_name_tag = soup.find("person:name")
            if not person_name_tag:
                return None

            # 提取person:name基本信息
            person_created_date_tag = person_name_tag.find("common:created-date")
            given_names_tag = person_name_tag.find("personal-details:given-names")
            family_name_tag = person_name_tag.find("personal-details:family-name")

            person_created_date = person_created_date_tag.string if person_created_date_tag else None
            given_name = given_names_tag.string if given_names_tag else None
            family_name = family_name_tag.string if family_name_tag else None

            # 性别识别 - 只处理女性
            if not given_name:
                return None
            
            gender_result = self.gender_detector.get_gender(given_name.split()[0])
            if gender_result not in ['female', 'mostly_female']:
                return None  # 只处理女性

            # 构建基准姓名 (given_name + family_name)
            if not family_name:
                return None
            base_name = f"{given_name} {family_name}".strip()

            # 提取other-name信息 - 可能有多个
            other_names_data = []
            other_name_tags = soup.find_all("other-name:other-name")
            
            for other_name_tag in other_name_tags:
                other_created_date_tag = other_name_tag.find("common:created-date")
                other_name_content_tag = other_name_tag.find("other-name:content")
                
                other_created_date = other_created_date_tag.string if other_created_date_tag else None
                other_name_content = other_name_content_tag.string if other_name_content_tag else None
                
                if other_created_date and other_name_content:
                    other_names_data.append({
                        'other_name_content': other_name_content,
                        'other_created_date': other_created_date
                    })

            if not other_names_data:
                return None

            return {
                'person_created_date': person_created_date,
                'given_name': given_name,
                'family_name': family_name,
                'base_name': base_name,
                'other_names_data': other_names_data,
                'gender': gender_result
            }

        except Exception as e:
            self.logger.debug(f"Error extracting name info: {e}")
            return None

    def is_name_change_candidate(self, name_info: Dict) -> Tuple[bool, Optional[List]]:
        """判断是否为姓名变化候选者"""
        try:
            from datetime import datetime, timedelta
            
            base_name = name_info['base_name']  # given_name + family_name
            person_created_date = name_info['person_created_date']
            other_names_data = name_info['other_names_data']
            family_name = name_info['family_name']
            given_name = name_info['given_name']

            if not all([base_name, person_created_date, other_names_data, family_name, given_name]):
                return False, None

            # 解析person:name的创建日期
            try:
                person_date = datetime.fromisoformat(person_created_date.replace('Z', '+00:00'))
            except:
                return False, None

            # 检查每个other-name
            for other_data in other_names_data:
                other_name_content = other_data['other_name_content']
                other_created_date = other_data['other_created_date']
                
                if not other_name_content or not other_created_date:
                    continue

                # 检查other-name中是否包含原始的姓氏或名字（关联性验证）
                if not (str(family_name).lower() in str(other_name_content).lower() or 
                       str(given_name).lower() in str(other_name_content).lower()):
                    continue

                # 解析other-name的创建日期
                try:
                    other_date = datetime.fromisoformat(other_created_date.replace('Z', '+00:00'))
                except:
                    continue

                # 检查时间差是否超过30天
                time_diff = abs((person_date - other_date).days)
                if time_diff < 30:
                    continue

                # 标准化姓名
                cleaned_base_name = str(base_name).strip().lower().replace('-', ' ').replace('.', '')
                cleaned_other_name = str(other_name_content).strip().lower().replace('-', ' ').replace('.', '')

                # 排除完全相同的姓名
                if cleaned_base_name.replace(' ', '') == cleaned_other_name.replace(' ', ''):
                    continue

                # 检查名字部分是否相同（确保是同一人）
                base_parts = cleaned_base_name.split(' ')
                other_parts = cleaned_other_name.split(' ')
                
                if len(base_parts) == 0 or len(other_parts) == 0:
                    continue
                    
                # 名字（first name）必须相同
                if base_parts[0] != other_parts[0]:
                    continue

                # 分析姓名变化模式
                base_surnames = base_parts[1:]  # 姓氏部分
                other_surnames = other_parts[1:]  # 姓氏部分
                
                if len(base_surnames) == 0 or len(other_surnames) == 0:
                    continue

                # 检查是否有共同姓氏
                common_surnames = set(base_surnames) & set(other_surnames)
                
                if len(common_surnames) > 0:
                    # 情况1：加中间名（有共同姓氏）
                    # 检查一个是另一个的子集
                    if (set(base_surnames).issubset(set(other_surnames)) and len(base_surnames) < len(other_surnames)) or \
                       (set(other_surnames).issubset(set(base_surnames)) and len(other_surnames) < len(base_surnames)):
                        
                        # 确定哪个是较早的日期
                        if person_date < other_date:
                            earlier_name = cleaned_base_name
                            earlier_date = person_created_date
                            later_name = cleaned_other_name
                            later_date = other_created_date
                        else:
                            earlier_name = cleaned_other_name
                            earlier_date = other_created_date
                            later_name = cleaned_base_name
                            later_date = person_created_date
                        
                        return True, [earlier_name, earlier_date, later_name, later_date]
                else:
                    # 情况2：改姓（无共同姓氏）
                    # 排除缩写情况
                    if len(base_surnames) == 1 and len(other_surnames) == 1:
                        if (len(base_surnames[0]) == 1 or len(other_surnames[0]) == 1) and \
                           base_surnames[0][0] == other_surnames[0][0]:
                            continue  # 跳过缩写情况
                        
                        return True, [cleaned_base_name, person_created_date, cleaned_other_name, other_created_date]

            return False, None

        except Exception as e:
            self.logger.debug(f"Error in name change detection: {e}")
            return False, None

    def process_single_file(self, file_path: Path, port_name: str) -> Optional[Dict]:
        """处理单个XML文件"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                soup = BeautifulSoup(file, 'xml')
                name_info = self.extract_name_info(soup)

                if not name_info:
                    return None

                is_change, change_data = self.is_name_change_candidate(name_info)
                if is_change and change_data:
                    path_tag = soup.find("common:path")
                    orcid_id = path_tag.string if path_tag else "unknown"
                    
                    return {
                        'port_name': port_name,
                        'id': orcid_id,
                        'person_name': change_data[0],
                        'person_date': change_data[1],
                        'other_name': change_data[2],
                        'other_date': change_data[3],
                        'gender': name_info['gender']
                    }
                return None

        except Exception as e:
            self.logger.debug(f"Error processing file {file_path}: {e}")
            return None

def process_file_chunk(args):
    """处理文件块的工作函数"""
    file_paths, port_name, process_id = args
    processor = ORCIDProcessor(CONFIG['base_path'], CONFIG['output_file'])
    results = []
    processed = 0
    
    for file_path in file_paths:
        try:
            result = processor.process_single_file(file_path, port_name)
            if result:
                results.append(result)
            processed += 1
            
            # 每处理1000000个文件报告一次进度
            if processed % 1000000 == 0:
                print(f"Process {process_id}: Processed {processed}/{len(file_paths)} files in {port_name}")
                
        except Exception as e:
            logging.error(f"Error in process {process_id} processing {file_path}: {e}")
    
    return results, processed, len(results)

def get_file_chunks(base_path: str, chunk_size: int = 1000) -> List[Tuple]:
    """获取文件分块信息"""
    base_path = Path(base_path)
    chunks = []
    chunk_id = 0
    
    for portfolio_folder in base_path.iterdir():
        if not portfolio_folder.is_dir():
            continue
            
        # 获取文件夹中的所有XML文件
        xml_files = list(portfolio_folder.glob("*.xml"))
        if not xml_files:
            xml_files = list(portfolio_folder.iterdir())  # 如果没有.xml扩展名
            
        # 将文件分块
        for i in range(0, len(xml_files), chunk_size):
            file_chunk = xml_files[i:i + chunk_size]
            chunks.append((file_chunk, portfolio_folder.name, chunk_id))
            chunk_id += 1
    
    return chunks

def write_results_batch(results_batch: List[Dict], output_file: str, write_header: bool = False):
    """批量写入结果到CSV文件"""
    mode = 'w' if write_header else 'a'
    with open(output_file, mode, newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['port_name', 'id', 'person_name', 'person_date', 
                                             'other_name', 'other_date', 'gender'])
        if write_header:
            writer.writeheader()
        writer.writerows(results_batch)

def signal_handler(signum, frame):
    """信号处理器"""
    print("\n收到中断信号，正在安全退出...")
    sys.exit(0)

def main():
    """主函数"""
    # 注册信号处理器
    signal.signal(signal.SIGINT, signal_handler)
    signal.signal(signal.SIGTERM, signal_handler)
    
    start_time = time.time()
    
    # 获取文件分块
    print("正在分析文件结构...")
    chunks = get_file_chunks(CONFIG['base_path'], CONFIG['chunk_size'])
    total_chunks = len(chunks)
    
    print(f"发现 {total_chunks} 个文件块")
    if total_chunks == 0:
        print("没有找到需要处理的文件")
        return
    
    # 初始化输出文件
    write_results_batch([], CONFIG['output_file'], write_header=True)
    
    # 统计变量
    total_processed = 0
    total_results = 0
    results_buffer = []
    
    try:
        # 使用进程池处理
        with ProcessPoolExecutor(max_workers=CONFIG['max_workers']) as executor:
            print(f"启动 {CONFIG['max_workers']} 个工作进程...")
            
            # 提交所有任务
            future_to_chunk = {executor.submit(process_file_chunk, chunk): i 
                              for i, chunk in enumerate(chunks)}
            
            # 处理完成的任务
            for future in as_completed(future_to_chunk):
                chunk_idx = future_to_chunk[future]
                try:
                    chunk_results, chunk_processed, chunk_found = future.result()
                    
                    total_processed += chunk_processed
                    total_results += chunk_found
                    results_buffer.extend(chunk_results)
                    
                    # 批量写入结果
                    if len(results_buffer) >= CONFIG['batch_size']:
                        write_results_batch(results_buffer, CONFIG['output_file'])
                        results_buffer = []
                    
                    # 进度报告
                    progress = (chunk_idx + 1) / total_chunks * 100
                    print(f"进度: {progress:.1f}% ({chunk_idx + 1}/{total_chunks}), "
                          f"已处理: {total_processed:,}, 发现变化: {total_results:,}")
                    
                except Exception as e:
                    print(f"处理块 {chunk_idx} 时出错: {e}")
    
    except KeyboardInterrupt:
        print("\n用户中断程序执行")
        return
    except Exception as e:
        print(f"程序执行出错: {e}")
        return
    finally:
        # 写入剩余结果
        if results_buffer:
            write_results_batch(results_buffer, CONFIG['output_file'])
    
    # 最终统计
    end_time = time.time()
    duration = end_time - start_time
    
    print("\n" + "="*60)
    print("处理完成!")
    print(f"总处理文件数: {total_processed:,}")
    print(f"发现姓名变化: {total_results:,}")
    print(f"处理时间: {duration/3600:.1f} 小时 ({duration/60:.1f} 分钟)")
    print(f"平均速度: {total_processed/duration:.0f} 文件/秒")
    print(f"结果已保存至: {CONFIG['output_file']}")
    print("="*60)

if __name__ == "__main__":   
    main()